In [ ]:
MODEL_NAME = 'usppmatch'
MODEL = 'usp'

In [ ]:
import sys
import os  
os.environ["TOKENIZERS_PARALLELISM"] = "false"

!rm -rf ./src
!ln -s ../input/{MODEL_NAME} ./src
if os.path.exists('/kaggle'):
  sys.path.append('/kaggle/input/pikachu2/utils')
  sys.path.append('/kaggle/input/pikachu2/third')
  sys.path.append('.')
!ls ../input

In [ ]:
!pip install -q icecream --no-index --find-links=file:///kaggle/input/icecream/ 
!pip install -q datasets --no-index --find-links=file:///kaggle/input/hug-datasets/ 

In [ ]:
from gezi.common import *
gezi.beijing_time_str()

In [ ]:
from src.preprocess import *
from src.postprocess import normalize
from src.eval import write_result

In [ ]:
INDEXES_LIST = [
    [0,1,2,3,4],
    [5],
    [6,7,8,9,10],
    [11],
    [12,13,14,15,16],
    [17],
    [18,19,20,21,22],
    [23],
    [24,25,26,27,28],
    [29],
    [30,31,32,33,34],
    [35],
    [36,37,38,39,40],
    [41],
]
# 5 folds deberta-v3-large
# WEIGHTS = [1]
# WEIGHTS = [
#            1.,1.,
#            0.4,0.8,
#            0.3,0.6,
#            0.2,0.4,
#           ]
## 8677 ensemble 
# WEIGHTS = [
#            1.,2.,
#            0.4,0.8,
#            0.3,0.6,
#            0.2,0.4,
#           ]
## 8682 ensemble
WEIGHTS = [
           0.6,1.2,
           0.3,0.6,
           0.2,0.4,
           0.2,0.4,
           0.4,0.8,
           0.1,0.2,
           0.1,0.2,
          ]

In [ ]:
def show_models(indexes, show=False):
  models = {}
  for i in indexes:
    model_dir = f'../input/{MODEL}-model{i}'
    model_path = open(f'{model_dir}/path.txt').readline().strip()
    models[i] = model_path
  ic(models)

def work(ensembler, indexes, weight=1.):
  for i in tqdm(indexes):
    model_dir = f'../input/{MODEL}-model{i}'
    bs = 128
    ofile = '../working/x.pkl'
    # with static_input_len and on some valid data to verify if gpu will not OOM, for debug only
#     command = f'python ./src/infer.py {model_dir} {bs} {ofile} 1 1'
    command = f'python ./src/infer.py {model_dir} {bs} {ofile}'
    ic(i, model_dir, bs, command, weight)
    os.system(command)
    x = gezi.load(ofile)
    ic(gezi.get_mem_gb())
    x['pred'] = normalize(x['pred'])
    ensembler.add(x, weight)

In [ ]:
gezi.init_flags()

for i, indexes in enumerate(INDEXES_LIST):
  ic(i, indexes, WEIGHTS[i])
  show_models(indexes)
    
ensembler = gezi.Ensembler(need_sort=True,includes=['pred'])
for i, indexes in tqdm(enumerate(INDEXES_LIST), total=len(INDEXES_LIST)):
  ic(i, indexes)
  show_models(indexes)
  work(ensembler, indexes, WEIGHTS[i])
x = ensembler.finalize()
x['pred'] = normalize(x['pred'])

In [ ]:
# will do normlaize in write_result
write_result(x, x['pred'], './submission.csv', is_infer=True)

In [ ]:
# x_test = x
# df = pd.read_csv(f'{FLAGS.root}/train.csv')
# df

In [ ]:
# ensemblers = [gezi.Ensembler(need_sort=True,includes=['bert_proba']) for _ in range(5)]
# xs = []
# n_models = int(len(INDEXES_LIST) / 2)
# for j in range(n_models):
#   if j != 1:
#     continue
#   for fold in range(5):
#     i = j * 6 + fold
#     root = f'../input/usp-model{i}'
#     x = gezi.load(f'{root}/valid.pkl')
#     x['fold'] = np.asarray([i] * len(x['id']))
#     x['bert_proba'] = normalize(x['pred'])
#     ic(len(x['bert_proba']))
#     ensemblers[fold].add(x, WEIGHTS[j * 2])
# xs = [ensembler.finalize() for ensembler in ensemblers]
# for x in xs:
#   x['bert_proba'] = normalize(x['bert_proba'])
# x = gezi.merge_array_dicts(xs)
# x

In [ ]:
# ic(len(x['id']), len(x['pred']), len(x['bert_proba']))

In [ ]:
# x['score'] = x['label']
# # x['bert_proba'] = normalize(x['bert_proba'])
# del x['text']
# del x['pred']
# del x['label']

In [ ]:
# x_train = pd.DataFrame(x)
# ic(x_train.columns)
# x_train

In [ ]:
# if 'score' in x_test:
#   del x_test['score']
# if 'fold' in x_test:
#   del x_test['fold']
# if 'label' in x_test:
#   del x_test['label']
# x_test['bert_proba'] = normalize(x_test['pred'])
# del x_test['pred']
# # x_test['context'] = np.asarray([m[a] for a in x_test['id']])
# x_test = pd.DataFrame(x_test)
# ic(x_test.columns)
# x_test

In [ ]:
# from src.stacking import run_stacking
# a, b = run_stacking(x_train, x_test)

In [ ]:
# df = pd.DataFrame({'id': b.id.values, 'score': b.stacking_proba.values})
# df

In [ ]:
# df.to_csv('./submission.csv', index=False)